In [78]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#import pandas_datareader as pdr
from datetime import datetime,timedelta
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense ,LSTM, Dropout,Bidirectional,TimeDistributed
from sklearn.model_selection import train_test_split, KFold
from tensorflow import expand_dims
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from os.path import join
from tensorflow import constant
from time import sleep
from prophet import Prophet
import tensorflow as tf

#import FinanceDataReader as fdr

In [79]:
data=pd.read_csv('data.csv')

In [137]:
gl=pd.read_csv('goldlee_added.csv')

In [82]:
data= data.drop('Unnamed: 0', axis = 1)

In [138]:
def convert_date(date_str):
  return pd.to_datetime(date_str, format="%Y%m%d").strftime("%Y-%m-%d")
data["일자"] =data["일자"].apply(convert_date)

In [139]:
def convert_date(date_str):
  return pd.to_datetime(date_str, format="%Y/%m/%d").strftime("%Y-%m-%d")
gl["일자"] =gl["일자"].apply(convert_date)

In [140]:
gl = gl[gl["일자"].isin(data["일자"].unique())]
gl.set_index('일자',inplace=True)

In [141]:
# prescaler
def getAOC(data):
  AOC = []
  close = data['종가']
  close_pre = close.shift(1)

  AOC.append(0)

  # aoc = (금일 종가 - 전일 종가) / 전일 종가
  for i in range(1, len(close)):
    aoc_value = ((close[i] - close_pre[i]) / close_pre[i]) * 100
    AOC.append(aoc_value)

  return AOC

#sort data by "종목명"
subject_Data=data.sort_values(by="종목코드",inplace=False)
subject_Data.head()
subject_Data.columns


pv_s_data=pd.pivot_table(subject_Data,values=['거래량', '시가', '고가', '저가', '종가'],index=['종목코드','일자'])
pv_s_data["AOC"] = getAOC(pv_s_data)

divided = pv_s_data.index.get_level_values('종목코드').unique()

dataframes = []

for i in divided:
    sliced_data = pv_s_data.loc[i]
    sliced_dataframe = sliced_data.reset_index()
    dataframes.append(sliced_dataframe)

sliced_dataframes=[]

for s in dataframes:
    end = s.pop('종가')
    sliced_dataframe = pd.merge(s, gl, left_on='일자', right_index=True, how='left')
    sliced_dataframe['종가'] = end
    sliced_dataframes.append(sliced_dataframe)

In [142]:
## sorting labels
labels=data[['종목코드','일자']]
labels = labels.sort_values(by=['종목코드','일자'])
labels=labels.drop('일자',axis=1)
labels=labels.drop_duplicates('종목코드')
data.sort_values(by=['종목코드',"일자"],inplace=True)

In [ ]:
label_list = labels['종목코드']

In [89]:
for i in sliced_dataframes:
    i['한국은행 기준금리'].fillna(i[i['일자'] == '2023-07-26']['한국은행 기준금리'].values[0], inplace = True)
    i['정부대출금금리'].fillna(i[i['일자'] == '2023-07-26']['정부대출금금리'].values[0], inplace = True)
    i['무역금융지원 프로그램대출금리'].fillna(i[i['일자'] == '2023-07-26']['무역금융지원 프로그램대출금리'].values[0], inplace = True)
    i['영세자영업자지원 프로그램대출금리'].fillna(i[i['일자'] == '2023-07-26']['영세자영업자지원 프로그램대출금리'].values[0], inplace = True)
    i['신성장·일자리지원 프로그램대출금리'].fillna(i[i['일자'] == '2023-07-26']['신성장·일자리지원 프로그램대출금리'].values[0], inplace = True)
    i['설비투자지원 프로그램대출금리'].fillna(i[i['일자'] == '2023-07-26']['설비투자지원 프로그램대출금리'].values[0], inplace = True)
    i['지방중소기업지원 프로그램대출금리'].fillna(i[i['일자'] == '2023-07-26']['지방중소기업지원 프로그램대출금리'].values[0], inplace = True)
    i['자금조정 대출금리'].fillna(i[i['일자'] == '2023-07-26']['자금조정 대출금리'].values[0], inplace = True)
    i['자금조정 예금금리'].fillna(i[i['일자'] == '2023-07-26']['자금조정 예금금리'].values[0], inplace = True)

In [144]:
for sliced in sliced_dataframes:
  sliced.drop('일자',axis=1,inplace=True)

## Model shells

## TEST **Shell**

## Predict **Shell**

In [92]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, LayerNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from os.path import join
import tensorflow as tf


class Transformer():
    
    def __init__(self, data, idx,day):
        self.data = data
        self.idx = idx
        self.time_steps = 60
        self.day=day
        
        
    def create_sequences(self, data):
        num_samples, num_features = data.shape
        sequences = []
        for i in range(num_samples - self.time_steps +1):
          sequences.append(data[i:i + self.time_steps, :])
        return np.array(sequences)

    def inverse_sequences(self,data_sequences):
        num_samples, num_steps, num_features = data_sequences.shape
        data = np.zeros((num_samples + self.time_steps - 1, num_features))
        for i in range(num_samples):
            data[i:i + self.time_steps, :] += data_sequences[i, :, :]
        data /= self.time_steps
        return data[:num_samples]
        #frquency domain inverse laplace transform
        
    def slicing_data(self):
        self.x_scaler = MinMaxScaler()
        self.y_scaler = MinMaxScaler()
        # Scaling features
        x_data = self.data[self.data.columns[:]]
        x_data = self.x_scaler.fit_transform(x_data)
        # Scaling target variable
        y_data = self.data[self.data.columns[:]]
        y_data = self.y_scaler.fit_transform(y_data)
        # validation set
        x_train_p, x_test, y_train_p, y_test = train_test_split(x_data, y_data, test_size=0.2, shuffle=False)
        x_train, x_val, y_train, y_val = train_test_split(x_train_p, y_train_p, test_size=0.25, shuffle=False)
        __t,x_final,__t,__t=train_test_split(x_data, y_data, test_size=60/len(y_data), shuffle=False)

        self.num_features = x_data.shape[1]
        self.num_samples=x_data.shape[0]

        self.x_train = self.create_sequences(x_train)
        self.x_test = self.create_sequences(x_test)
        self.x_val = self.create_sequences(x_val)
        self.x_final = self.create_sequences(x_final)

        self.y_train=self.create_sequences(y_train)
        self.y_test=self.create_sequences(y_test)
        self.y_val=self.create_sequences(y_val)

    def transformer_encoder_block(self, inputs):
        # Multi-Head Self-Attention
        attention = tf.keras.layers.MultiHeadAttention(key_dim=self.num_features // 8, num_heads=8)(inputs, inputs)
        attention = Dropout(0.1)(attention)
        x = LayerNormalization(epsilon=1e-6)(inputs + attention)
        
        # Feed Forward Neural Network
        x = Dense(4 * self.num_features, activation="relu")(x)
        x = Dropout(0.1)(x)
        x = Dense(self.num_features, activation="relu")(x)
        x = LayerNormalization(epsilon=1e-6)(x + inputs)
        return x
    
    def transformer_model(self):
        # Input layer
        inputs = Input(shape=(self.time_steps, self.num_features))
        # Transformer Encoder
        x = inputs
        for _ in range(4):  # Transformer Encoder Block의 반복 횟수 (하이퍼파라미터)
            x = self.transformer_encoder_block(x)
        # Output layer
        x = Dense(self.num_features, activation='relu')(x)
        model = Model(inputs, x, name="transformer_model")
        return model


    def train_model(self):
        self.model = self.transformer_model()
        model_save_path = '/content/drive/MyDrive/2023_1st_vac/KRX_modelings/best_model/Transformer2/{}/'.format(self.day)
        self.filename = join(model_save_path, 'checkpoint_{}_{}.ckpt'.format(self.day,self.idx))
        checkpoint = ModelCheckpoint(self.filename, save_weights_only=True, save_best_only=True, monitor='val_loss', verbose=0)
        earlystopping = EarlyStopping(monitor='val_loss', patience=100)
        self.model.compile(loss='mean_squared_error', optimizer=Adam(learning_rate=0.001))  # 학습률 (하이퍼파라미터)
        self.history = self.model.fit(self.x_train, self.y_train, epochs=1, batch_size=128, validation_data=(self.x_val, self.y_val), shuffle=False, callbacks=[checkpoint, earlystopping], verbose=0)

    def get_gap_by_test(self):
        self.model.load_weights(self.filename)
        pred = self.model.predict(self.x_test,verbose=0)
        rescaled_pred = self.y_scaler.inverse_transform(self.inverse_sequences(pred))
        rescaled_real = self.y_scaler.inverse_transform(self.inverse_sequences(self.y_test))

        gaps = []

        for p, r in zip(rescaled_pred, rescaled_real):
            gap = np.abs(p - r)
            gaps.append(gap)

        avg_gap = np.mean(gaps)

        return avg_gap

    def return_val(self):
        self.slicing_data()
        self.train_model()
        self.model.load_weights(self.filename)  

        next_input_data = np.copy(self.x_final)
        for day in range(1, 16):
            pred_day = self.model.predict(np.expand_dims(next_input_data[-1], axis=0), verbose=0)
            next_input_data = np.concatenate((next_input_data, pred_day), axis=0)

        next_15_days_data = next_input_data[-15:]
        next_15_days_data = self.y_scaler.inverse_transform(self.inverse_sequences(next_15_days_data))

        columns = self.data.columns.tolist()
        ret_df = pd.DataFrame(next_15_days_data, columns=columns)

        prices = []
        for end_price in ret_df.iloc[:, -1]:
            prices.append(end_price)

        gap = self.get_gap_by_test()

        return prices, gap


'''
하이퍼파라미터 추천:
-->  논문 이상치  instanced
Transformer Encoder Block의 반복 횟수: 4
Multi-Head Self-Attention에서 key_dim: 입력 특성 수의 1/8 크기로 추천 (예: num_features=100 이라면 key_dim=12)
학습률 (learning_rate): 0.001 (Adam optimizer를 사용하는 경우 일반적으로 사용되는 값)
Dropout 비율: 0.1 (적절한 정도의 regularization을 위한 값)
Transformer Encoder Block의 뉴런 수 (Dense layer의 뉴런 수): 입력 특성 수의 몇 배로 할지 지정 (예: 입력 특성 수가 100이면 4 * num_features=400 정도로 지정)
'''

'\n하이퍼파라미터 추천:\n-->  논문 이상치  instanced\nTransformer Encoder Block의 반복 횟수: 4\nMulti-Head Self-Attention에서 key_dim: 입력 특성 수의 1/8 크기로 추천 (예: num_features=100 이라면 key_dim=12)\n학습률 (learning_rate): 0.001 (Adam optimizer를 사용하는 경우 일반적으로 사용되는 값)\nDropout 비율: 0.1 (적절한 정도의 regularization을 위한 값)\nTransformer Encoder Block의 뉴런 수 (Dense layer의 뉴런 수): 입력 특성 수의 몇 배로 할지 지정 (예: 입력 특성 수가 100이면 4 * num_features=400 정도로 지정)\n'

In [94]:
idx=0

pred_list=[]
gap_acc=[]
out_labels=[]

In [95]:
for c in sliced_dataframes:
  print(labels.iloc[idx])
  print("{}번째 종목코드".format(idx))
  trans_model=Transformer(data=c,idx=idx,day='230727_1')
  pred,gap = trans_model.return_val()
  sleep(1)
  pred_list.append(pred)
  gap_acc.append(gap)
  idx+=1
  #to prevent model corrupting

종목코드    A000020
Name: 502, dtype: object
0번째 종목코드
종목코드    A000040
Name: 118, dtype: object
1번째 종목코드
종목코드    A000050
Name: 243, dtype: object
2번째 종목코드
종목코드    A000070
Name: 760, dtype: object
3번째 종목코드
종목코드    A000080
Name: 1803, dtype: object
4번째 종목코드
종목코드    A000100
Name: 1328, dtype: object
5번째 종목코드
종목코드    A000120
Name: 16, dtype: object
6번째 종목코드
종목코드    A000140
Name: 1804, dtype: object
7번째 종목코드
종목코드    A000150
Name: 503, dtype: object
8번째 종목코드
종목코드    A000180
Name: 840, dtype: object
9번째 종목코드
종목코드    A000210
Name: 33, dtype: object
10번째 종목코드
종목코드    A000220
Name: 1320, dtype: object
11번째 종목코드
종목코드    A000230
Name: 1412, dtype: object
12번째 종목코드
종목코드    A000240
Name: 1823, dtype: object
13번째 종목코드
종목코드    A000250
Name: 776, dtype: object
14번째 종목코드
종목코드    A000270
Name: 298, dtype: object
15번째 종목코드
종목코드    A000300
Name: 416, dtype: object
16번째 종목코드
종목코드    A000320
Name: 346, dtype: object
17번째 종목코드
종목코드    A000370
Name: 1903, dtype: object
18번째 종목코드
종목코드    A000390
Name: 787, dtype: ob

ValueError: Input contains infinity or a value too large for dtype('float64').

In [155]:
for c in sliced_dataframes:
  print(labels.iloc[idx-10])
  print("{}번째 종목코드".format(idx))
  trans_model=Transformer(data=c,idx=idx,day='230727_1')
  pred,gap = trans_model.return_val()
  sleep(1)
  pred_list.append(pred)
  #gap_acc.append(gap)
  idx+=1
  #to prevent model corrupting

종목코드    A363280
Name: 1692, dtype: object
2000번째 종목코드


## Gap 큰 것만 다시 학습

In [ ]:
value = {}
value['종목코드'] = labels
value['종가'] = pred_list
value['오차'] = gap_acc

value = pd.DataFrame(value)

In [ ]:
top_gap_indices = value.sort_values(by='Gap', ascending=False).index[:200]

for idx in top_gap_indices:
    c = sliced_dataframes[idx]
    print(value.loc[idx], '종목코드')
    print("{}번째 종목코드".format(idx))
    trans_model = Transformer(data = c, idx = idx, day = '230727_1')
    pred, gap = trans_model.return_val()
    sleep(1)
    value.loc[idx, '종가'] = pred
    value.loc[idx, '오차'] = gap

In [ ]:
labels = value['종목코드']
pred_list = value['종가']
gap_acc = value['오차']

## 순위 정하기
1. 그냥 이익률 or 차액만 따지기

In [ ]:
len(pred_list)

In [164]:
past_list = []
for i in sliced_dataframes:
  past_list.append(i['종가'][493])

In [165]:
pred_15 = []
for i in pred_list:
    pred_15.append(i[-1])

In [166]:
def subtract_arrays(arr1, arr2):
    if len(arr1) != len(arr2):
        raise ValueError("Arrays must have the same length")

    result = []
    for i in range(len(arr1)):
        result.append(arr1[i] - arr2[i])
    return result

def divide_arrays(arr1, arr2):
    if len(arr1) != len(arr2):
        raise ValueError("Arrays must have the same length")

    result = []
    for i in range(len(arr1)):
        result.append(arr1[i] / arr2[i])
    return result

In [167]:
benefit_sub = subtract_arrays(pred_15, past_list)
benefit_div = subtract_arrays(pred_15, past_list)

In [169]:
label_lis

['종목코드']

In [184]:
df_1=pd.DataFrame({
        '종목코드': label_list,
        '이익': benefit_sub
    }
)

In [188]:
df_2=pd.DataFrame({
        '종목코드': label_list,
        '이익': benefit_div
    }
)

In [185]:
df_1.sort_values(by='이익', ascending = False, inplace = True)
df_1.drop(columns=['이익'], inplace = True)
df_1['순위'] = list(range(1, 2001))

In [189]:
df_2.sort_values(by='이익', ascending = False, inplace = True)
df_2.drop(columns=['이익'], inplace = True)
df_2['순위'] = list(range(1, 2001))

In [186]:
df_1.to_csv('{}.csv'.format("230729_v0.1"),index=False)

In [190]:
df_2.to_csv('{}.csv'.format("230729_v0.2"),index=False)

2. 샤프지수

In [191]:
# 투자 기간과 무위험 수익률 설정
trading_days = 15
risk_free_rate = 0.035

# 주어진 데이터에 따라 샤프 지수 계산
total_return = []
for i in pred_list:
    initial_asset = i[0]  # 초기 자산 (1일의 종가)
    final_asset = i[-1]  # 최종 자산 (15일의 종가)
    total_return.append((final_asset - initial_asset) / initial_asset)

daily_returns = []
for i in pred_list:
    daily_returns.append(i[-1] - i[-2] / i[-2])

annualized_daily_returns = []
for i in pred_list:
    annualized_daily_returns.append([daily_return * 250 for daily_return in daily_returns])

daily_return_volatility = []
for i in range(len(pred_list)):
    daily_return_volatility.append(np.std(annualized_daily_returns[i], ddof=1))

annualized_total_return = []
for i in range(len(pred_list)):
    annualized_total_return.append(total_return[i] * (250 / len(pred_list)))

sharpe_ratio = []
for i in range(len(pred_list)):
    sharpe_ratio.append((annualized_total_return[i] - risk_free_rate) / daily_return_volatility[i])

In [193]:
df_3=pd.DataFrame({
        '종목코드': label_list,
        '샤프지수': sharpe_ratio
    }
)

In [194]:
df_3.sort_values(by='샤프지수', ascending = False, inplace = True)
df_3.drop(columns=['샤프지수'], inplace = True)
df_3['순위'] = list(range(1, 2001))

In [195]:
df_3

,종목코드,순위
1714,A131760,1
289,A001570,2
1360,A093230,3
11,A013720,4
212,A002710,5
...,...,...
1587,A183300,1996
1959,A010690,1997
23,A083660,1998
1010,A101390,1999


In [196]:
df_3.to_csv('{}.csv'.format("230729_v0.3"),index=False)